Import Crunchbase DataBase with all the companies around the world

In [1]:
# Import modules
from pymongo import MongoClient
import pandas as pd
import json as js

In [2]:
client = MongoClient("mongodb://localhost/companies")
client = MongoClient('localhost', 27017) # Default connection params
client = MongoClient('mongodb://localhost:27017') # Mongourl style

db = client.get_database()

ConfigurationError: No default database defined

In [3]:
list(db["companies"].find()[:1])

NameError: name 'db' is not defined